# Movie Reviews and Bag-of-Words Modelling

🎯 The goal of this challenge is to play with the ***Bag-of-words*** modelling of texts.

✍️ In the following dataset, we have $2000$ reviews classified either as _"positive"_ or _"negative"_.

In [3]:
import pandas as pd

data = pd.read_csv("https://wagon-public-datasets.s3.amazonaws.com/05-Machine-Learning/10-Natural-Language-Processing/movie_reviews.csv")
data.head()

,target,reviews
0,neg,"plot : two teen couples go to a church party ,..."
1,neg,the happy bastard's quick movie review \ndamn ...
2,neg,it is movies like these that make a jaded movi...
3,neg,""" quest for camelot "" is warner bros . ' firs..."
4,neg,synopsis : a mentally unstable man undergoing ...


In [4]:
data.shape

(2000, 2)

## 1. Preprocessing

❓ **Question (Cleaning Text)** ❓

- Write a function `preprocessing` that will clean a sentence and apply it to all our reviews. It should:
    - remove whitespace
    - lowercase characters
    - remove numbers
    - remove punctuation
    - tokenize
    - lemmatize
- You can store the cleaned reviews into a column called `cleaned_reviews`.
- Do not remove stopwords in this challenge, we will explain why in the section `3. N-gram modelling`

In [5]:
import string
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer

def preprocessing(sentence):
    sentence=sentence.strip()
    sentence=sentence.lower()
    sentence=''.join(char for char in sentence if not char.isdigit())
    for punctuation in string.punctuation:
        sentence = sentence.replace(punctuation, '') 
    sentence=" ".join(word_tokenize(sentence))
    sentence_vb=WordNetLemmatizer().lemmatize(sentence, pos = "v")
    sentence=WordNetLemmatizer().lemmatize(sentence_vb, pos = "n")
    return sentence

In [7]:
# Clean reviews
data["cleaned_rewievs"]=data[["reviews"]].applymap(preprocessing)
data=data.drop(columns="reviews")
data

,target,cleaned_rewievs
0,neg,plot two teen couples go to a church party dri...
1,neg,the happy bastards quick movie review damn tha...
2,neg,it is movies like these that make a jaded movi...
3,neg,quest for camelot is warner bros first feature...
4,neg,synopsis a mentally unstable man undergoing ps...
...,...,...
1995,pos,wow what a movie its everything a movie can be...
1996,pos,richard gere can be a commanding actor but hes...
1997,pos,glorystarring matthew broderick denzel washing...
1998,pos,steven spielbergs second epic film on world wa...


❓ **Question (LabelEncoding)**❓

LabelEncode your target and store it into a column called `"target_encoded"`

In [8]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(data.target)
data["target_encoded"]=le.transform(data.target)
data

,target,cleaned_rewievs,target_encoded
0,neg,plot two teen couples go to a church party dri...,0
1,neg,the happy bastards quick movie review damn tha...,0
2,neg,it is movies like these that make a jaded movi...,0
3,neg,quest for camelot is warner bros first feature...,0
4,neg,synopsis a mentally unstable man undergoing ps...,0
...,...,...,...
1995,pos,wow what a movie its everything a movie can be...,1
1996,pos,richard gere can be a commanding actor but hes...,1
1997,pos,glorystarring matthew broderick denzel washing...,1
1998,pos,steven spielbergs second epic film on world wa...,1


In [9]:
# Quick check
data.head()

,target,cleaned_rewievs,target_encoded
0,neg,plot two teen couples go to a church party dri...,0
1,neg,the happy bastards quick movie review damn tha...,0
2,neg,it is movies like these that make a jaded movi...,0
3,neg,quest for camelot is warner bros first feature...,0
4,neg,synopsis a mentally unstable man undergoing ps...,0


## 2. Bag-of-Words Modelling

❓ **Question (NaiveBayes with unigrams)** ❓

Using `cross_validate`, score a Multinomial Naive Bayes model trained on a Bag-of-Words representation of the texts.

In [10]:
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
from sklearn.model_selection import cross_validate
from sklearn.pipeline import make_pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import recall_score


# Pipeline vectorizer + Naive Bayes
pipeline_naive_bayes = make_pipeline(
    CountVectorizer(), 
    MultinomialNB()
)

# Cross-validation
cv_results = cross_validate(pipeline_naive_bayes, X=data["cleaned_rewievs"], y=data["target_encoded"], cv = 5, scoring = ["recall"])
average_recall = cv_results["test_recall"].mean()
np.round(average_recall,2)

0.8

## 3. N-gram Modelling

👀 Remember that we asked you not to remove stopwords. Why? 

👉 We will train the Naive Bayes model with bigrams. Hence, in sentence like "I do not like coriander", it is important to scan the bigram "do not" to detect negativity in this sentence for example.

❓ **Question (NaiveBayes with bigrams)** ❓

Using `cross_validate`, score a Multinomial Naive Bayes model trained on a 2-gram Bag-of-Words representation of the texts.

In [13]:
vectorizer = CountVectorizer(ngram_range = (2,2))
naivebayes = MultinomialNB()

X_bow = vectorizer.fit_transform(data.cleaned_rewievs)

cv_nb = cross_validate(
    naivebayes,
    X_bow,
    data.target_encoded,
    scoring = "accuracy"
)

round(cv_nb['test_score'].mean(),2)

0.84

🏁 Congratulations! Now, you know how to train a Naive Bayes model on vectorized texts.

💾 Don't forget to `git add/commit/push` your notebook...

🚀 ... and move on to the next challenge!